In [1]:
import sys
import ujson
import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# For displaying google maps
import gmplot
from IPython.display import IFrame

# Second (better?) google map, only inline so far
import gmaps

# For summary stats
import scipy
from scipy import stats

# For testing a subset of the data
from copy import deepcopy

pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read config from local file
GMAP_API_KEY = None

sys.path.append('/Users/patrickmulrooney/Desktop/')
import capstone_config

_file = capstone_config._INPUT_FILE
GMAP_API_KEY = capstone_config._GMAP_API_KEY

gmaps.configure(api_key=GMAP_API_KEY)

print _file

/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed_no_outliers.json


In [3]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

In [4]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')


time: 62.1 ms


# Working with data already parsed from LoadData - 1 notebook
## Load parsed data

In [5]:
#time: 12min 44s
# Set via config file
# Full dataset
#_file = "/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed.json"
# Full dataset, outliers removed
#_file = "/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed_no_outliers.json"
# Partial dataset
#_file = "/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed_10k.json"
# Partial dataset, outliers removed
#_file = "/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed_no_outliers_10k.json"

endoHR = []

with open(_file, 'r') as f:
    endoHR += ujson.load(f)

time: 10min 51s


In [6]:
print "Length of import: %s"%(len(endoHR))

Length of import: 251774
time: 8.6 ms


In [ ]:
## Used between cells

# List of all the exercise types in the dataset
exercise_types = {}

# Keys in each record
all_keys = []


series_keys = { True: ['timestamp', 'heart_rate', 'altitude', 'latitude', 'longitude', 'speed' ], \
               False: ['timestamp', 'heart_rate', 'altitude', 'latitude', 'longitude'] }

In [ ]:
# See what keys still remain.
sports = {}
count = 0
for _endoHR in endoHR:
    all_keys += _endoHR.keys()
    _sport = _endoHR[ 'sport' ]
    try:
        sports[_sport] += 1
    except:
        sports[_sport] = 1
        
    if _sport not in exercise_types.keys():
        exercise_types[_sport] = count
        count += 1

# Drop dupes
all_keys = set( all_keys )

sports2 = []

for k,v in sports.iteritems():
    sports2.append((k,v))

print "What keys remain?"
pp.pprint( all_keys )

print ""
print ""

print "What sports?"
pp.pprint( sorted(sports2, key=itemgetter(1), reverse=True) )

In [ ]:
seriesLength = []

for _endoHR in log_progress(endoHR, every=5):

    # For debugging purposes, get a record of length of series for histogram
    _seriesLen = len(_endoHR['series'])
    seriesLength.append(_seriesLen)
    
plt.hist(seriesLength, bins=50, log=True)
plt.show()

### Notes
Looks like there must be some sort of limit with the HR stuff of 500 records per exercise, or the api not returning more than that. Need to look at this further.

### Get deltas of timestamps and speed in series. 


In [ ]:

for _endoHR in log_progress(endoHR, every=1):
    _max_time_delta = 0
    
    _timestamps = [item[0] for item in _endoHR['series']]
    _endoHR['end_start_delta'] = max(_timestamps) - min(_timestamps)

    for i,_cur in enumerate(_timestamps[:-2]):
        _next = _timestamps[i+1]
        _diff = _next - _cur
        if _diff > _max_time_delta:
            _max_time_delta = _diff
    
    _endoHR['max_time_delta'] = _max_time_delta
    
    if _endoHR['speed_included']:
        _max_speed_delta = 0
        _max_speed = 0
        
        _speeds = [item[5] for item in _endoHR['series']]

        for i,_cur in enumerate(_speeds[:-2]):
            _next = _speeds[i+1]
            _diff = _next - _cur
            
            if _diff > _max_speed_delta:
                _max_speed_delta = _diff
                
            if _cur > _max_speed:
                _max_speed = _cur
    
        _endoHR['max_speed'] = _max_speed
        _endoHR['max_speed_delta'] = _max_speed_delta


In [ ]:
# See how long 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

_esd = sorted([(item['end_start_delta']/60) for item in endoHR])

print pd.Series(_esd).describe()

# Based on summary stats, lets only keep those less than 6 hours. Beyond that is crazy.
#_esd = [_i for _i in _esd if _i < 360]
_esd = [_i for _i in _esd if _i < 2880]


plt.hist(_esd,bins=200,log=True)
plt.show()

## Figure out where these people are.

In [ ]:
#time: 9min 12s
lat_longs = []

for _endoHR in log_progress(endoHR, every=1):
    _series = _endoHR['series'][0]
    lat_longs.append([_series[3], _series[4], _endoHR['sport']])

In [ ]:
#time: 7.94 s
# Write the lat longs to file, need to figure out how to map. Probably google maps and kml, try basemap below.

target = open("/tmp/lat_longs.txt", 'w')
for i in lat_longs:
    target.write("%s,%s\n"%(i[0],i[1]))
    
target.close()

In [ ]:
lats = [ _i[0] for _i in lat_longs ]
lons = [ _i[1] for _i in lat_longs ]
sport_type = [ exercise_types[_i[2]] for _i in lat_longs ]

In [ ]:
# Americas
plt.figure(figsize=(12,12))
m = Basemap(width=15000000,height=20000000,resolution='i',projection='stere', lat_ts=0,lat_0=10,lon_0=-80.)
x, y = m(lons,lats)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966',lake_color='#99ffff',zorder=0)
m.scatter(x,y,1, marker='o')
m.drawcountries()
plt.show()

In [ ]:
# Europe, Africa, Asia, etc
plt.figure(figsize=(12,12))
m = Basemap(width=30000000,height=20000000,resolution='i',projection='stere', lat_ts=0,lat_0=10,lon_0=80.)
x, y = m(lons,lats)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966',lake_color='#99ffff',zorder=0)
m.scatter(x,y,1, marker='o')
m.drawcountries()
plt.show()

In [ ]:
#US
lat=40
lon=-100

plt.figure(figsize=(12,12))
m = Basemap(width=5000000,height=3000000,resolution='h',projection='stere', lat_ts=0,lat_0=lat,lon_0=lon)
x, y = m(lons,lats)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966',lake_color='#99ffff',zorder=0)
m.scatter(x,y,1, marker='o')
m.drawcountries()
plt.show()

In [ ]:
#Europe
lat=50
lon=10

plt.figure(figsize=(12,12))
m = Basemap(width=5000000,height=3000000,resolution='h',projection='stere', lat_ts=0,lat_0=lat,lon_0=lon)
x, y = m(lons,lats)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966',lake_color='#99ffff',zorder=0)
m.scatter(x,y,1, marker='o')
m.drawcountries()
plt.show()

In [ ]:

lats_rounded = [ float("{0:.4f}".format(_i)) for _i in lats ]
lons_rounded = [ float("{0:.4f}".format(_i)) for _i in lons ]
sport_type = [ exercise_types[_i[2]] for _i in lat_longs ]

# # declare the center of the map, and how much we want the map zoomed in
# gmap = gmplot.GoogleMapPlotter(0, 0, 2)
# # plot heatmap
# gmap.heatmap(lats_rounded, lons_rounded)

# gmap.draw("/tmp/workout_heatmap.html")

m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=zip(lats_rounded, lons_rounded))
m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 90
heatmap_layer.point_radius = 3
m


#### Notes
* Really popular in Copenhagen. Endomundo was founded in Copenhagen.
* Really popular in Bangkok. No idea why

## Figure out who works out the most, and how much

In [ ]:
#time: 9min 51s
_uid_freq = {}
for _endoHR in log_progress(endoHR, every=5):
    try:
        _uid_freq[_endoHR['userId']] += 1
    except:
        _uid_freq[_endoHR['userId']] = 1
        
_uid_freq2 = []

for k,v in _uid_freq.iteritems():
    _uid_freq2.append((k,v))

print "Top 100 users"
pp.pprint( sorted(_uid_freq2, key=itemgetter(1), reverse=True)[:100] )

In [ ]:
plt.hist(sorted([_i[1] for _i in _uid_freq2]),bins=100,log=True)
plt.show()

### What sports did the top 100 most active users do?

In [ ]:
#time: 9min 51s
_top_participants = [ _i[0] for _i in sorted(_uid_freq2, key=itemgetter(1), reverse=True)[:100]]

_uid_freq_detail = {}
for _endoHR in log_progress(endoHR, every=1):
    if not _endoHR['userId'] in _uid_freq_detail.keys():
        _uid_freq_detail[_endoHR['userId']] = {}
    if _endoHR['sport'] in _uid_freq_detail[_endoHR['userId']].keys():
        _uid_freq_detail[_endoHR['userId']][_endoHR['sport']] += 1
    else:
        _uid_freq_detail[_endoHR['userId']][_endoHR['sport']] = 1
        
_uid_freq_detail2 = []

for k,v in _uid_freq_detail.iteritems():
    _uid_freq_detail2.append((k,v))

sorted(_uid_freq_detail2, key=lambda x: sum(x[1].values()) , reverse=True)[:100]

In [ ]:
# Lets figure out where user 4997910 lives / works.
# So many bike commutes probably will be able top guess based on heat map

_user = []

for _i in endoHR:
    if _i['userId'] == 3714939:
        _user.append(_i['series'])


In [ ]:
_user = [ item for sublist in _user for item in sublist]

In [ ]:
print len(_user)
_user[:10]

In [ ]:
#_user_lat = [round(_i[3],3) for _i in _user]
#_user_long = [round(_i[4],3) for _i in _user]
_user_lat_long = [(round(_i[3],3),round(_i[4],3)) for _i in _user]

In [ ]:
_weighted_user_lat_long_dict = {}
for _i in _user:
    _tmp_lat_long = (round(_i[3],3),round(_i[4],3))
    
    if _tmp_lat_long not in _weighted_user_lat_long_dict:
        _weighted_user_lat_long_dict[_tmp_lat_long] = 1
    else:
        _weighted_user_lat_long_dict[_tmp_lat_long] += 1

_weighted_user_lat_long = []
for k, v in _weighted_user_lat_long_dict.iteritems():
    _weighted_user_lat_long.append((k[0],k[1],v))

In [ ]:
# gmap = gmplot.GoogleMapPlotter(0, 0, 2)
# # plot heatmap
# gmap.heatmap(_user_lat, _user_long)

# gmap.draw("/tmp/workout_heatmap_3714939.html")

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.WeightedHeatmap(data=_weighted_user_lat_long)
m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 90
heatmap_layer.point_radius = 3
m

### Was able to determine where this user likely lives, and where they likely work

### Lets try the top 5 users

In [ ]:

# _top_users = [ _j[0] for _j  in sorted(_uid_freq_detail2, key=lambda x: sum(x[1].values()) , reverse=True)[:10]]

# for _j in log_progress(_top_users):
#     _user = []
#     for _i in endoHR:
#         if _i['userId'] == _j:
#             _user.append(_i['series'])

#     # Flatten
#     _user = [ item for sublist in _user for item in sublist]
#     _user_lat = [_i[3] for _i in _user]
#     _user_long = [_i[4] for _i in _user]

#     gmap = gmplot.GoogleMapPlotter(0, 0, 2)
#     # plot heatmap
#     gmap.heatmap(_user_lat, _user_long)
#     # map markers
#     #gmap.scatter(_user_lat, _user_long, '#3B0B39', size=40, marker=False)
#     #gmap.scatter(_user_lat, _user_long, 'k', marker=True)
#     # Not sure..
#     #gmap.plot(_user_lat, _user_long, 'cornflowerblue', edge_width=10)


#     gmap.draw("/tmp/workout_heatmap_%s.html"%_j)

## Lets get summary stats of series time

In [ ]:
endoHR_stats = []

for _endoHR in log_progress(endoHR, every=1):
    _summary = { 'average_diff': 0, 'median_diff': [], 'delta': 0, }
    
    _timestamps = [item[0] for item in _endoHR['series']]
    
    # only concern ourselves with those more than 60 timestamps in the activity.
    if len(_timestamps) < 60:
        continue
    
    _summary['delta'] = _timestamps[-1] - _timestamps[0]
    
    
    for _i, _timestamp in enumerate(_timestamps[:-2]):
        _diff = _timestamps[_i+1] - _timestamps[_i]
        _summary['average_diff'] += _diff
        _summary['median_diff'].append(_diff)
        
    try:
        _summary['average_diff'] = _summary['average_diff'] / len(_timestamps)
        _summary['median_diff'] = sorted(_summary['median_diff'])[len(_timestamps)/2]
    except:
        print len(_timestamps)
    endoHR_stats.append(_summary)
    


In [ ]:
endoHR_stats
print "Average"
print scipy.stats.describe([_i['average_diff'] for _i in endoHR_stats])

print ""
print "Median"
print scipy.stats.describe([_i['median_diff'] for _i in endoHR_stats])


## Heartrate analysis

In [ ]:
_heart_rates = [ item[1] for sublist in endoHR for item in sublist['series']]

In [ ]:
plt.hist(sorted(_heart_rates),bins=100, log=True)
plt.show()

In [ ]:
# heart rate summary stats
print scipy.stats.describe(_heart_rates)

#### Looks like there is a fair amount of garbage in the heart rate data. Min = -706, Max = 1001. Lets set some limits.

from: https://www.reddit.com/r/askscience/comments/1opolp/what_is_the_fastest_the_average_human_heart_could/

```
The absolute refractory period of cardiac muscle is between 250 and 300 ms. That means cardiac contractions are physiologically limited to 200-240 beats per minute.
```

#### So lets set a max of 250 and min of 40. Below that likely to be error.

In [ ]:
_bad_heart_rates = [ _i for _i in _heart_rates if _i not in range(40,250)]
_heart_rates = [ _i for _i in _heart_rates if _i in range(40,250)]

In [ ]:
print scipy.stats.describe(_heart_rates)

In [ ]:
plt.hist(sorted(_heart_rates),bins=100, log=True)
plt.show()

#### Bad heart rate numbers

In [ ]:
plt.hist(sorted(_bad_heart_rates),bins=100, log=True)
plt.show()

## Speed analysis

In [ ]:
_speed = [ item[5] for sublist in endoHR for item in sublist['series'] if sublist['speed_included'] == True]

In [ ]:
plt.hist(sorted(_speed),bins=100, log=True)
plt.show()

In [ ]:
print scipy.stats.describe(_speed)

#### No speed below zero, but seems unlikely someone got up to 235 MPH (assuming it is MPH, need to check. 235KM/h is just as unlikely). Lets look at it by sport 

In [ ]:
for _i in exercise_types.keys():
    _speed = [ item[5] for sublist in endoHR for item in sublist['series'] if sublist['speed_included'] == True and sublist['sport'] == _i]
    print "------"
    print _i
    print len(_speed)
    if len(_speed) == 0:
        continue
    plt.hist(sorted(_speed),bins=100, log=True)
    plt.title(_i)
    plt.show()

### Altitude

In [ ]:
_altitude = [ item[2] for sublist in endoHR for item in sublist['series'] ]

In [ ]:
plt.hist(sorted(_altitude),bins=100, log=True)
plt.show()

In [ ]:
print scipy.stats.describe(_altitude)

In [ ]:
_high_altitude_locations = [item[3:5] for sublist in endoHR for item in sublist['series'] if item[2] > 3500]

In [ ]:
_high_altitude_locations[:5]

In [ ]:
# # declare the center of the map, and how much we want the map zoomed in
# gmap = gmplot.GoogleMapPlotter(0, 0, 2)
# # plot heatmap
# gmap.scatter([_i[0] for _i in _high_altitude_locations ],[_i[1] for _i in _high_altitude_locations ])

# gmap.draw("/tmp/high_altitude_workout_locations.html")

_lat_longs = zip([_i[0] for _i in _high_altitude_locations ],[_i[1] for _i in _high_altitude_locations ])

m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=_lat_longs)
m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 90
heatmap_layer.point_radius = 3
m

In [ ]:
for _i in endoHR:
    if len([1 for item in _i['series'] if item[2] > 5000]) > 0:
        print _i['gender'], _i['id'], _i['sport'], _i['url'], _i['userId']

## Sanity check latitude and longitude

#### Values should only between -90 / 90 & -180 / 180

In [ ]:
_latitude = [ item[3] for sublist in endoHR for item in sublist['series'] ]
_longitude = [ item[4] for sublist in endoHR for item in sublist['series'] ]

print scipy.stats.describe(_latitude)
print scipy.stats.describe(_longitude)

#### Looks good

## Check San DIego users

In [28]:
sd_events = []
for _endoHR in log_progress(endoHR):
    _mid_point = []
    
    if len(_endoHR['series']) < 10:
        continue
        
    _series = _endoHR['series']
    _point = _series[len(_series) / 2]
    
    _lat = _point[3]
    _long = _point[4]
    
    if _lat >= 32.4 and _lat <= 33.0 and _long <= -116.9 and _long >= -117.5:
        sd_events.append(_endoHR)

time: 1.39 s


In [29]:
print "Found %s records near SD"%len(sd_events)

Found 26 records near SD
time: 1.49 ms


In [30]:
print set([ _i['userId'] for _i in sd_events ])

set([3276737, 504709, 8888545, 7430612, 458903, 4226525])
time: 1.52 ms


In [31]:
_lat_longs = []
for _endoHR in sd_events:
    _series = _endoHR['series']
    
    [ _lat_longs.append([_i[3], _i[4]]) for _i in _series]

time: 73.5 ms


In [23]:
len(_lat_longs)

9024

time: 2.99 ms


In [32]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=_lat_longs)
m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 5
m

time: 284 ms
